<a href="https://colab.research.google.com/github/Arditata/DEEPLRN/blob/main/cektext.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Install dan Import Library**

---



In [1]:
!pip install transformers torch scikit-learn gradio

In [2]:
import gradio as gr
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import random

# **2. Masukan Model dan Tokenizer**

In [3]:
# Load BERT model
model_name = 'sentence-transformers/bert-base-nli-mean-tokens'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# **3. Mendefenisikan Fungsi Embedding dan Similarity**

In [4]:
# Function to generate embeddings with BERT
def generate_embeddings(text):
    tokens = tokenizer(text, return_tensors='pt', padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**tokens)
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

# Function to calculate similarity using cosine similarity
def calculate_similarity(text1, text2):
    embeddings1 = generate_embeddings(text1)
    embeddings2 = generate_embeddings(text2)
    similarity = cosine_similarity(embeddings1, embeddings2)
    return similarity[0][0]

# **4. Jalankan Chatbot**

In [5]:
# Chatbot function
def chatbot(user_input, comparison_text):
    if not user_input or not comparison_text:
        return "Please enter both input and comparison text."

    similarity_percentage = calculate_similarity(user_input, comparison_text) * 100
    response_text = f"The similarity between your input and the comparison text is: {similarity_percentage:.2f}%"

    # Generate a random follow-up response
    follow_up_responses = [
        "Do you want to compare another pair of texts?",
        "Would you like to continue the conversation?",
        "Feel free to ask me about anything else!"
    ]
    follow_up = random.choice(follow_up_responses)

    return f"{response_text}\n{follow_up}"

# Create Gradio interface
iface = gr.Interface(
    fn=chatbot,
    inputs=["text", "text"],
    outputs="text",
    title="Chatbot Similarity Checker",
    description="Enter two texts, and I'll calculate their similarity using BERT."
)

# Launch Gradio app
iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://23ed027648150c3691.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
